In [354]:
import glob
# This file should run basically insta fast. Bad coding for sure 
genSavePath = "F:/PythonProjects/NietoExcercise-1/SavedResults/"
day = "05" # Which month and days result you want to visualize in table
month = 11
repetitionValue = 11
repetitionName = "udrlBC3CVTest"
repetitionString = f"{repetitionValue}{repetitionName}"
curSavePath = f"{genSavePath}{day}-{month}-{repetitionString}/"
savedResults = glob.glob(curSavePath + "savedBestSeed*")

skipSubjects = []
onlySignificant = True
minAcc = 0.6
print(len(savedResults))
print(len(savedResults)/9)

72
8.0


In [355]:
import pandas as pd
from array import ArrayType
from requests import delete
from tabulate import tabulate
from prettytable import PrettyTable as pt
import numpy as np


nameDict = dict()
nrOfNames = 0
kernelDict = dict()
nrOfKernel = 0
nrOfCombination = 0
currSeed = 0
currSubj = 0
combinationList = []
combinationResult = []


results = []
for path in savedResults:
    results.append(np.load(path, allow_pickle=True))

subjectAmount = 10
for x in range(subjectAmount):
    combinationResult.append([])

nrOfSeedsPerSubject = np.zeros(subjectAmount)
for s, subject in enumerate(results):
    currSubj = 0
    currSeed = 0
    subjectBestArray = np.empty([len(subject)], dtype=object)

    currSeed = subject[0]
    currSubj = subject[1]

    if currSubj in skipSubjects:
        continue
    nrOfSeedsPerSubject[currSubj] += 1

    for aResults in subject[2]:
        for aResults2 in aResults:
            # If the np array comes from svm results
            if len(aResults2) < 6:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                if ares > 0.98:
                    ares = 0.99
                akernel = aResults2[2]
                aC = round(aResults2[3], 2)

                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [nameDict[aName], kernelDict[akernel], aC] not in combinationList:
                    combinationList.append([nameDict[aName], kernelDict[akernel], aC])
                    nrOfCombination += 1

                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aC]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                    
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])

                    # If the np array comes from NN results
            else:

                aName = aResults2[0]
                ares = aResults2[1]

                if type(ares) is tuple:
                    ares = ares[0]
                akernel = aResults2[2]
                aLz = round(aResults2[3], 2)
                areg = round(aResults2[4], 2)
                adrp = round(aResults2[5], 2)
                if aName not in nameDict:
                    nameDict[aName] = nrOfNames
                    nrOfNames += 1
                if akernel not in kernelDict:
                    kernelDict[akernel] = nrOfKernel
                    nrOfKernel += 1
                if [
                    nameDict[aName],
                    kernelDict[akernel],
                    aLz,
                    areg,
                    adrp,
                ] not in combinationList:
                    combinationList.append(
                        [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                    )
                    nrOfCombination += 1
                # combResult = []
                combNr = combinationList.index(
                    [nameDict[aName], kernelDict[akernel], aLz, areg, adrp]
                )
                if any(combNr in sublist for sublist in combinationResult[currSubj]):
                    sublist = np.array(combinationResult[currSubj])
                  
                    index = int(np.where(sublist[:, 0] == combNr)[0])

                    combinationResult[currSubj][index][1] = combinationResult[currSubj][
                        index
                    ][1] * (1 - (1 / nrOfSeedsPerSubject[currSubj])) + ares * (
                        1 / nrOfSeedsPerSubject[currSubj]
                    )

                else:

                    combinationResult[currSubj].append([combNr, ares])


# Making table

# headers = [f"{x[0]}:{x[1]}:{x[2]}:" for x in combinationList]
headers = np.arange(nrOfCombination)
subjectResultArray = np.empty([10, nrOfCombination], dtype=object)
subjectNrList = []

comboAverage = np.zeros([1, nrOfCombination])
comboTotal = np.zeros([1, nrOfCombination])
nrOfSubjects = np.zeros(nrOfCombination)

topTen = []
topThreeAllSubjects = []
lowestTopRes = [0, 0]
for subNr, cResult in enumerate(combinationResult):
    topThreeThisSubject = []
    resArray = np.zeros([nrOfCombination])
    if len(cResult) != 0:
        for res in cResult:
            if onlySignificant == True:
                if res[1] < minAcc:
                    continue
            comboAverage[0, res[0]] = comboAverage[0, res[0]] + res[1]
            comboTotal[0, res[0]] = comboTotal[0, res[0]] + res[1]
            nrOfSubjects[res[0]] += 1
            subjectResultArray[subNr, res[0]] = res[1]

            if len(topTen) < 10 or topTen[0][0] < res[1]:

                if len(topTen) == 10:
                    topTen.reverse()
                    topTen.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topTen.append([res[1], subNr, dataF, kernel, cVal])
                topTen.sort()

            
            
            if len(topThreeThisSubject) < 2 or topThreeThisSubject[0][0] < res[1]:

                if len(topThreeThisSubject) == 2:
                    topThreeThisSubject.reverse()
                    topThreeThisSubject.pop()

                dataF = list(nameDict.keys())[combinationList[res[0]][0]]
                kernel = list(kernelDict.keys())[combinationList[res[0]][1]]
                cVal = combinationList[res[0]][2]
                topThreeThisSubject.append([res[1], subNr, dataF, kernel, cVal])
                topThreeThisSubject.sort()
    if len(topThreeThisSubject)>0:
        topThreeAllSubjects.append(topThreeThisSubject)
# Print the table!
comboAverage = comboAverage / nrOfSubjects
print(comboAverage.shape)
print(subjectResultArray.shape)
pd.set_option("display.max_columns", None)

# For loop through zip of headers and columns,
# If column has all values == None, Remove column and header.
subjectResultArray = np.array(subjectResultArray)
emptyCols = []
atleastOneEmpty = False
for hcol, in zip(headers):
    if all(subjectResultArray[:,hcol] == None):
        atleastOneEmpty = True
        emptyCols.append(hcol)
if atleastOneEmpty: 
    subjectResultArray = np.delete(subjectResultArray, np.array(emptyCols),1)
    headers = np.delete(headers, np.array(emptyCols),0)
    comboAverage = np.delete(comboAverage, np.array(emptyCols),1)
    comboTotal = np.delete(comboTotal, np.array(emptyCols),1)
    nrOfSubjects = np.delete(nrOfSubjects, np.array(emptyCols),0)


table = pd.DataFrame(subjectResultArray, columns=headers)
display(table)




c:\Users\Luna\anaconda3\envs\mentalab3\lib\site-packages\ipykernel_launcher.py:188: RuntimeWarning: invalid value encountered in true_divide


(1, 1407)
(10, 1407)


6       7         8         9         10        11        12    \
0      None    None      None      None      None      None      None   
1      None    None      None  0.715625  0.715625       0.7   0.61875   
2      None    None      None  0.731771  0.744792  0.747396  0.630208   
3      None    None      None  0.663194  0.670139  0.677083      None   
4      None    None      None  0.721354  0.710938  0.742188      None   
5  0.617188    None      None  0.760417  0.700521  0.744792  0.614583   
6      None    None      None  0.684659  0.661932  0.673295  0.639205   
7      None    None      None  0.708333  0.726562  0.729167  0.601563   
8    0.6625  0.6125  0.703125    0.6875  0.696875  0.703125   0.66875   
9      None    None      None  0.677083  0.671875  0.692708      None   

       13        14       18        19        20        21        23    \
0      None      None     None      None      None      None      None   
1   0.65625  0.671875     None      None      None      None      None   
2  0.669271  0.664063     None  0.638021  0.627604      None      None   
3  0.694444      None     None      None      None      None      None   
4      None  0.643229     None      None      None      None      None   
5  0.648438  0.713542     None  0.666667   0.65625      None  0.638021   
6  0.642045  0.627841     None      None      None      None      None   
7      None  0.632813    0.625  0.661458  0.606771      None      None   
8   0.68125  0.671875  0.63125  0.665625      None  0.671875    0.7125   
9      None      None     None      None      None      None      None   

       30        31        32        36        37        38        39    \
0      None      None      None      None      None      None      None   
1  0.690625   0.69375  0.690625      None  0.615625     0.675      None   
2  0.721354   0.71875  0.734375      None  0.658854  0.617188  0.622396   
3  0.701389  0.704861   0.71875  0.607639  0.607639      None      None   
4  0.765625  0.747396  0.757813      None  0.617188  0.635417  0.604167   
5  0.723958  0.703125  0.716146      None  0.664063  0.619792  0.638021   
6  0.698864  0.678977  0.696023      None      None      None      None   
7  0.690104  0.684896  0.716146      None  0.648438  0.627604  0.619792   
8  0.715625  0.696875  0.728125   0.63125   0.68125  0.646875  0.634375   
9   0.71875  0.729167  0.723958  0.648438  0.651042  0.604167      None   

       40        41       45    46        47        48        49        50    \
0      None      None     None  None      None      None      None      None   
1      None  0.609375     None  None      None  0.728125  0.728125  0.696875   
2      None  0.645833     None  None      None  0.734375  0.752604  0.742188   
3      None      None     None  None      None  0.663194  0.670139  0.677083   
4      None  0.609375     None  None      None  0.716146  0.716146  0.747396   
5      None  0.638021     None  None      None  0.768229  0.697917  0.755208   
6      None      None     None  None      None    0.6875  0.659091  0.676136   
7  0.648438  0.614583     None  None      None   0.71875  0.713542  0.726563   
8  0.646875  0.665625  0.65625  0.65  0.653125  0.709375  0.696875  0.703125   
9      None      None     None  None      None  0.669271  0.669271  0.684896   

       51        52        53        57        58        59        60    \
0      None      None      None      None      None      None      None   
1   0.63125    0.6625  0.690625      None      None      None      None   
2  0.645833  0.669271  0.653646      None  0.606771      None      None   
3      None    0.6875  0.604167      None      None      None      None   
4  0.601563      None  0.640625      None  0.619792  0.606771      None   
5  0.674479  0.705729  0.726562  0.700521  0.716146  0.700521  0.645833   
6  0.642045  0.642045  0.633523      None      None      None      None   
7  0.611979      None  0.640625  0.627604  0.640625  0.601562      None   
8  0.6

In [356]:
print("Top ten highest accuracies")

topTen.reverse()
for top in topTen:
    print(
            f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
        )


Top ten highest accuracies
Accuracy 0.79 : subject 5, feature:welchDatacn3-dataGCV-BC--fftDataBC, kernellinear, C:0.5
Accuracy 0.79 : subject 5, feature:dataHRBC-fftDataBC--fftData, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 5, feature:dataGCV-BCcn3-dataHRBC--fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 5, feature:gaussianDatacn3BC-dataHRBC--fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 5, feature:dataHRBC-fftDataBC--welchData, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 5, feature:dataGCV-BCcn3-fftDatacn3--fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 5, feature:gaussianDataBC-dataHRBC--fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 5, feature:gaussianData-dataHRBC--fftDataBC, kernelsigmoid, C:2.5
Accuracy 0.79 : subject 5, feature:fftDatacn3-dataGCV-BC--fftDataBC, kernellinear, C:0.5
Accuracy 0.78 : subject 5, feature:dataGCV-BC-fftDataBC--fftData, kernellinear, C:0.5


In [357]:
print("Top ten highest accuracies")
#del(topThreeThisSubject[0])
#len(topThreeAllSubjects[0])
for subTop in topThreeAllSubjects:
    subTop.reverse()
topThreeAllSubjects.sort(reverse=True)
for subTop in topThreeAllSubjects:
    print("")
    #subTop.reverse()
    for top in subTop:
        print(
                f"Accuracy {round(top[0],2)} : subject {top[1]}, feature:{top[2]}, kernel{top[3]}, C:{top[4]}"
            )

Top ten highest accuracies

Accuracy 0.79 : subject 5, feature:welchDatacn3-dataGCV-BC--fftDataBC, kernellinear, C:0.5
Accuracy 0.79 : subject 5, feature:dataHRBC-fftDataBC--fftData, kernelsigmoid, C:2.5

Accuracy 0.78 : subject 4, feature:fftDatacn3BC-fftDataBC--welchData, kernelsigmoid, C:2.5
Accuracy 0.78 : subject 4, feature:fftDatacn3BC-fftDataBC--welchData, kernelrbf, C:2.5

Accuracy 0.77 : subject 2, feature:dataGCV-BCcn3-fftDatacn3--fftDataBC, kernellinear, C:0.5
Accuracy 0.77 : subject 2, feature:gaussianDatacn3BC-fftDatacn3--fftDataBC, kernelsigmoid, C:2.5

Accuracy 0.77 : subject 9, feature:dataGCV-BCcn3-gaussianDatacn3--fftDatacn3BC, kernellinear, C:0.5
Accuracy 0.77 : subject 9, feature:dataGCV-BCcn3-fftDatacn3BC--dataHRCV-BC, kernellinear, C:0.5

Accuracy 0.77 : subject 8, feature:fftDatacn3-dataGCV-BC--fftDataBC, kernellinear, C:0.5
Accuracy 0.75 : subject 8, feature:fftDatacn3BC-dataGCV-BC--fftDataBC, kernellinear, C:0.5

Accuracy 0.76 : subject 1, feature:gaussianDatac

In [358]:
subjAv = np.zeros(subjectResultArray.shape[0])
for s, subj in enumerate(subjectResultArray):
    subjAv[s] = 0
    if all(subj == None):
        continue
    subjAv[s] = np.mean(subj[np.where(subj != None)])
print(subjAv)
pd.set_option("display.max_columns", None)
table = pd.DataFrame(np.array(comboAverage), columns=headers)
display(table)

print(
    f"Max average accuracy {np.max(np.array(comboAverage[np.where(np.isnan(comboAverage)==False)]))}"
)
maxVal = np.max(np.array(comboAverage[np.where(np.isnan(comboAverage) == False)]))
maxInd = int(np.where(comboAverage == maxVal)[1][0])
print(headers[maxInd])
print(nrOfSubjects[maxInd])
print(list(nameDict.keys())[combinationList[maxInd][0]])
print(list(kernelDict.keys())[combinationList[maxInd][1]])
print(f"C: {combinationList[maxInd][2]}")


[0.         0.67776379 0.68633168 0.67240487 0.6873683  0.69699456
 0.66477669 0.66552563 0.68510911 0.69232158]


,6,7,8,9,10,11,12,13,14,18,19,20,21,23,30,31,32,36,37,38,39,40,41,45,46,47,48,49,50,51,52,53,57,58,59,60,62,69,70,71,75,76,77,78,79,80,82,83,84,85,86,87,88,89,93,94,95,96,98,105,106,107,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,212,219,220,221,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,254,255,257,258,259,260,261,263,264,265,266,267,268,269,273,274,275,279,280,281,282,283,284,285,286,287,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,316,317,318,319,320,321,322,323,327,328,329,330,331,332,339,340,341,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,453,454,455,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,488,489,491,492,493,494,495,496,497,498,499,500,501,502,503,507,508,509,513,514,515,516,517,518,519,520,521,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,556,558,559,560,561,562,563,570,571,572,574,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,644,651,652,653,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,686,687,689,690,691,692,693,694,695,696,697,698,699,700,701,705,706,707,711,712,713,714,715,716,717,718,719,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,811,812,813,814,816,817,818,819,820,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,868,870,871,872,874,875,876,877,878,879,880,881,882,883,884,885,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127,11

Max average accuracy 0.7292648709315377
402
9.0
dataHRCV-BC-welchData--fftData
linear
C: 0.5


In [359]:
x = 0
for names in nameDict.keys():
    print(names)
    x +=1
    print(x)

-fftData
1
-welchData
2
-dataHRCV-BC
3
-fftDataBC
4
-dataHRBC
5
-gaussianData
6
-gaussianDataBC
7
-dataGCV-BC
8
-fftDatacn3
9
-welchDatacn3
10
-fftDatacn3BC
11
-gaussianDatacn3
12
-gaussianDatacn3BC
13
-dataGCV-BCcn3
14
welchData--fftData
15
dataHRCV-BC--fftData
16
fftDataBC--fftData
17
dataHRBC--fftData
18
gaussianData--fftData
19
gaussianDataBC--fftData
20
dataGCV-BC--fftData
21
fftDatacn3--fftData
22
welchDatacn3--fftData
23
fftDatacn3BC--fftData
24
gaussianDatacn3--fftData
25
gaussianDatacn3BC--fftData
26
dataGCV-BCcn3--fftData
27
dataHRCV-BC--welchData
28
fftDataBC--welchData
29
dataHRBC--welchData
30
gaussianData--welchData
31
gaussianDataBC--welchData
32
dataGCV-BC--welchData
33
fftDatacn3--welchData
34
welchDatacn3--welchData
35
fftDatacn3BC--welchData
36
gaussianDatacn3--welchData
37
gaussianDatacn3BC--welchData
38
dataGCV-BCcn3--welchData
39
fftDataBC--dataHRCV-BC
40
dataHRBC--dataHRCV-BC
41
gaussianData--dataHRCV-BC
42
gaussianDataBC--dataHRCV-BC
43
dataGCV-BC--dataHRCV-BC
4

In [360]:
table = pd.DataFrame(np.array(comboTotal), columns=headers)
display(table)

maxVal2 = np.max(np.array(comboTotal[np.where(np.isnan(comboTotal) == False)]))

for maxInd2 in np.where(comboTotal == maxVal2)[1]:

    # maxInd2 = int(np.where(comboTotal == maxVal2)[1])
    print(headers[maxInd2])
    print(nrOfSubjects[maxInd2])
    print(
        f"Max total significant accuracy then averaged {maxVal2/nrOfSubjects[maxInd2]} "
    )

    print(list(nameDict.keys())[combinationList[maxInd2][0]])
    print(list(kernelDict.keys())[combinationList[maxInd2][1]])
    print(f"C: {combinationList[maxInd2][2]} ")


,6,7,8,9,10,11,12,13,14,18,19,20,21,23,30,31,32,36,37,38,39,40,41,45,46,47,48,49,50,51,52,53,57,58,59,60,62,69,70,71,75,76,77,78,79,80,82,83,84,85,86,87,88,89,93,94,95,96,98,105,106,107,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,212,219,220,221,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,254,255,257,258,259,260,261,263,264,265,266,267,268,269,273,274,275,279,280,281,282,283,284,285,286,287,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,316,317,318,319,320,321,322,323,327,328,329,330,331,332,339,340,341,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,453,454,455,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,488,489,491,492,493,494,495,496,497,498,499,500,501,502,503,507,508,509,513,514,515,516,517,518,519,520,521,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,556,558,559,560,561,562,563,570,571,572,574,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,644,651,652,653,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,686,687,689,690,691,692,693,694,695,696,697,698,699,700,701,705,706,707,711,712,713,714,715,716,717,718,719,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,811,812,813,814,816,817,818,819,820,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,868,870,871,872,874,875,876,877,878,879,880,881,882,883,884,885,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127,11

402
9.0
Max total significant accuracy then averaged 0.7292648709315377 
dataHRCV-BC-welchData--fftData
linear
C: 0.5 


In [361]:
# maxAcc = np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])])
# print(f"Max single accuracy position {np.where(subjectResultArray == maxAcc)}")
# print(np.max([res for res in np.array(
#     subjectResultArray[subjectResultArray != None])]))
